In [1]:
import numpy as np
import math
import Bio

In [ ]:
score_matrix = np.array([[-math.inf, -1, -1, -1, -1],
                         [-1, 1, -1, -1, -1],
                         [-1, -1, 1, -1, -1],
                         [-1, -1, -1, 1, -1],
                         [-1, -1, -1, -1, 1]])

testo = "GCATGCU"
testt = "GATTACA"

In [ ]:
score_matrix = np.array([[-math.inf, -1, -1, -1, -1],
                         [-1, 1, -1, -1, -1],
                         [-1, -1, 1, -1, -1],
                         [-1, -1, -1, 1, -1],
                         [-1, -1, -1, -1, 1]])

testo = "GCATGCU"
testt = "GATTACA"